In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"

In [15]:

def get_data(url, first=False):
    print(url)
    url_2 = url.replace("BSheet", "IncSta").replace("bao-cao-tai-chinh-","ket-qua-hoat-dong-kinh-doanh-")
    url_3 = url.replace("BSheet", "CashFlow").replace("bao-cao-tai-chinh-","luu-chuyen-tien-te-gian-tiep-")
    
    data1 = pd.read_html(url)
    # get name colums
    names = data1[2]
    cls_name = names.iloc[0].tolist()
    cls_name[0] = "Tiêu chí"
    cls_name = cls_name[:5]
#     print(cls_name)
    
    # get data from 1 to 9
        # get data
    df1 = data1[3]
    df1 = df1.iloc[[0,13,14,15,16,17,21],[0,1,2,3,4]]
    df1.columns = cls_name
    
    # lấy tài sản cố định vô hình
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    
    # print(soup.prettify()) # print the parsed data of html
    table = soup.find("table", id="tableContent")

    rows = table.find_all('tr')

    tscd = []
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        if len(cols) > 0 and "Tài sản cố định vô hình" in cols[0]:
            tscd = cols
    a = [int(x.replace(",","")) if len(x) > 0 else 0 for x in tscd[1:5]]
    b = [tscd[0]]
    b = b + a
    s = pd.Series(b, index=df1.columns)
    df1 = df1.append(s, ignore_index=True)
    df1 = df1.iloc[[0,7,1,2,3,4,5,6]]
    
    # get data from 10 to 13
    data2 =  pd.read_html(url_2)
    df2 = data2[3]
    df2 = df2.iloc[[0,2,3,18],[0,1,2,3,4]]
    df2.columns = cls_name
    
    # geta data 14 (25 gian tiep 8 truc tiep)
    data3 =  pd.read_html(url_3)
    df3 = data3[3]
    df3 = df3.iloc[[25],[0,1,2,3,4]]
    df3.columns = cls_name

    df = pd.concat([df1,df2, df3], axis=0)
    df = df.replace(np.nan, 0)
#     print(df)
#     return df
    df.columns = df.columns.str.strip()
    df.iloc[:, [1,2,3,4]] = df.iloc[:, [1,2,3,4]].astype('float')
    
    df = df.reset_index(drop=True)
    df.columns = cls_name
    return df
    


In [16]:
url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"
# url1 = url.replace("/G36/", f"/{name}/")
# df = get_head(url1)
df = get_data(url)
df

https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn


,Tiêu chí,2017,2018,2019,2020
0,I - TÀI SẢN NGẮN HẠN,3.861165e+12,3.630369e+12,3.646422e+12,2.917963e+12
1,2.3. Tài sản cố định vô hình,3.159354e+10,3.160728e+10,3.160090e+10,3.150120e+10
2,Tổng cộng tài sản,6.134816e+12,5.932213e+12,5.837864e+12,4.938301e+12
3,I - NỢ PHẢI TRẢ,5.113967e+12,4.881540e+12,4.830293e+12,3.880177e+12
4,1. Nợ ngắn hạn,3.480813e+12,3.376828e+12,3.547450e+12,2.702949e+12
5,2. Nợ dài hạn,1.633154e+12,1.504712e+12,1.282843e+12,1.177228e+12
6,II - VỐN CHỦ SỞ HỮU,1.020850e+12,1.050673e+12,1.007571e+12,1.058124e+12
7,Tổng cộng nguồn vốn,6.134816e+12,5.932213e+12,5.837864e+12,4.938301e+12
8,1. Doanh thu bán hàng và cung cấp dịch vụ,3.575170e+12,3.084749e+12,1.794618e+12,2.308851e+12
9,3. Doanh thu thuần về bán hàng và cung cấp dịc...,3.575170e+12,3.084749e+12,1.794618e+12,2.308851e+12


In [17]:
def get_eps(name="G36"):
#     url = f"https://data.masvn.com/Controls/Report/StockFinance_Quarter.aspx?language=vi&scode={name}&page=1"
    url = f"https://data.masvn.com/Controls/Report/StockFinance_Year.aspx?language=vi&scode={name}&page=1"
    data = pd.read_html(url)
    df = data[0]
    data_eps = df.iloc[[13,17], [0,2,3,4,5]].reset_index(drop=True)
    data_eps.columns = data_eps.iloc[0]
    data_eps = data_eps.drop([0])
    
    filter_col = [col for col in data_eps if any( k in col for k in ['2017', '2018', '2019', '2020', '2021'])]
    select_col = ["CHỈ TIÊU CƠ BẢN"] + filter_col
    data_eps = data_eps[select_col].reset_index(drop=True)
    return data_eps


In [18]:
def get_company_name(url):

    html_content = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    name_company = soup.find("title").text.strip()
    for r in (("Báo cáo tài chính", ""), ("|", ""), (": ",""), ("CafeF.vn","")):
        name_company = name_company.replace(*r)
        
    name_company = " ".join(name_company.split())
    name_company = name_company.split(maxsplit=1)[-1]
    return name_company

In [19]:
def wrap(url, name, index=4):
    print(f"Crawl data name {name}")
    df = get_data(url)
    df = df.loc[:,~df.columns.duplicated()]
    df_eps = get_eps(name)
    df_eps=df_eps.replace('-',0)
    df_eps = df_eps.loc[:,~df_eps.columns.duplicated()]

    s = df.iloc[2]
    n = ["Tổng tải sàn bình quân "]
    n.append(s.iloc[1])
    for i in range(2, len(s)):
        n.append(int((s.iloc[i]+s.iloc[i-1]+1)/2))

    new = pd.Series(n, index=df.columns)
    df1 = df.append(new, ignore_index=True)
    df2 = df1.iloc[[0,1,2,13,3,4,5,6,7,8,9,10,11,12]]
    df2 = df2.reset_index(drop=True)


    df3 = df_eps.reset_index(drop=True)


    eps = [df3.iloc[0,0]]
    year = 2016
    for i in range(1,len(df3.columns)):
        year = year + 1
        while str(year) not in df3.columns[i] and year < 2022:
            eps.append(0)
            year = year + 1

        eps.append(int(df3.iloc[0,i]))

    eps = eps + [0] * 5
    k = len(df2.columns)
    eps = eps[0:k]
    df2 = df2.append(pd.Series(eps, index=df2.columns), ignore_index=True)
    
    df2.to_csv(f"data_{index}/{name}/{name}.csv", mode='a', index_label="stt")
#         my_file = Path("output.xlsx")
#         if my_file.is_file():
#             with pd.ExcelWriter('output.xlsx', mode='a') as writer:
#                 data.to_excel(writer, sheet_name=name)
#         else:
#             data.to_excel("output.xlsx", sheet_name=name)
#     return data

In [20]:
ck = pd.read_csv('ck1.csv')
ck = ck.replace(np.nan, 1)

In [21]:
def get_folder(index=1):
    ck = pd.read_excel('ck1.xls', sheet_name="1")
    fails = []
    for i in range(241,len(ck)):
        name = ck.loc[i, 'name']
        print(f"Folder {index} - Crawl data {i}: {name}")
        Path(f"data_{index}/{name}").mkdir(parents=True, exist_ok=True)
        
        
        url1 = url.replace("/G36/", f"/{name}/")
        company_name = get_company_name(url1)
        # assign data of lists.  
        data = {'Name': [company_name]}  

        # Create DataFrame  
        df = pd.DataFrame(data)
        df.to_csv(f"data_{index}/{name}/{name}.csv", mode='a')

#         print(f"{i}: {name}")
#         wrap(url1, name, index)
        try:
            wrap(url1, name, index)
        except:
            print(f"faile name: {name}")
            fails.append(name)
        with open(f"fails_{index}.txt", 'w') as f:
            for item in fails:
                f.write(f"'{item}', ")


In [ ]:
url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"
get_folder(5)

Folder 5 - Crawl data 241: VSH
Crawl data name VSH
https://s.cafef.vn/bao-cao-tai-chinh/VSH/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 242: OPC
Crawl data name OPC
https://s.cafef.vn/bao-cao-tai-chinh/OPC/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 243: LIX
Crawl data name LIX
https://s.cafef.vn/bao-cao-tai-chinh/LIX/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 244: PIT
Crawl data name PIT
https://s.cafef.vn/bao-cao-tai-chinh/PIT/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 245: SPM
Crawl data name SPM
https://s.cafef.vn/bao-cao-tai-chinh/SPM/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 246: TDP
Crawl data name TDP
https://s.cafef.vn/bao-cao-tai-chinh/TDP/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Fold

Folder 5 - Crawl data 291: TCI
Crawl data name TCI
https://s.cafef.vn/bao-cao-tai-chinh/TCI/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 292: VST
Crawl data name VST
https://s.cafef.vn/bao-cao-tai-chinh/VST/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 293: VGI
Crawl data name VGI
https://s.cafef.vn/bao-cao-tai-chinh/VGI/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 294: AAS
Crawl data name AAS
https://s.cafef.vn/bao-cao-tai-chinh/AAS/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 295: NAB
Crawl data name NAB
https://s.cafef.vn/bao-cao-tai-chinh/NAB/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 296: CAD
Crawl data name CAD
https://s.cafef.vn/bao-cao-tai-chinh/CAD/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Fold

Folder 5 - Crawl data 341: SCJ
Crawl data name SCJ
https://s.cafef.vn/bao-cao-tai-chinh/SCJ/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
Folder 5 - Crawl data 342: DDN
Crawl data name DDN
https://s.cafef.vn/bao-cao-tai-chinh/DDN/BSheet/2021/0/0/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn


In [ ]:
get_folder(2)
get_folder(3)

In [12]:
start = 2017
end = 2021
url = "https://s.cafef.vn/bao-cao-tai-chinh/G36/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn"
fails = []
for i in range(len(ck)):
    name = ck.loc[i,'name']
    Path(f"data_4/{name}").mkdir(parents=True, exist_ok=True)
    
    
    url1 = url.replace("/G36/", f"/{name}/")
    company_name = get_company_name(url1)
    # assign data of lists.  
    data = {'Name': [company_name]}  

    # Create DataFrame  
    df = pd.DataFrame(data)
    df.to_csv(f"data_4/{name}/{name}.csv", mode='a')
    
    print(f"{i}: {name}")
    wrap(url1, name)
#     try:
#         wrap(url1, name)
#     except:
#         print(f"faile name: {name}")
#         fails.append(name)



0: DS3
Crawl data name DS3
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2018/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/DS3/BSheet/2017/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
1: SZB
Crawl data name SZB
https://s.cafef.vn/bao-cao-tai-chinh/SZB/BSheet/2021/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SZB/BSheet/2020/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SZB/BSheet/2019/4/0/0/luu-chuyen-tien-te-truc-tiep-tong-cong-ty-36-ctcp.chn
https://s.cafef.vn/bao-cao-tai-chinh/SZB/B

In [43]:
for name in fails[2:]:
    url1 = url.replace("/G36/", f"/{name}/")
    company_name = get_company_name(url1)
    # assign data of lists.  
    data = {'Name': [company_name]}  
#     # Create DataFrame  
#     df = pd.DataFrame(data)
#     df.to_csv(f"data/{name}/{name}.csv", mode='a')
#     wrap(url1, name)
#     try:
#         wrap(url1, name)
#     except:
#         print(f"faile name: {name}")
#         fails.append(name)

In [42]:
print(fails)

['NHH', 'VPG', 'VSC', 'HHP', 'HTN', 'KOS', 'ADG', 'VNG', 'THG', 'ASG', 'BIC', 'PGI', 'CLL', 'VNS', 'NHT', 'AST', 'VPS', 'LM8', 'SPP', 'FDG', 'NTB', 'NHP', 'CLG', 'OIL', 'LMH', 'ATA', 'VAB', 'BOT', 'DVN', 'D82', 'HTT', 'NDF', 'ILA', 'MSR', 'VLF', 'HDO', 'AMS', 'SSH', 'PVE', 'APF', 'PRT', 'VNH', 'L44', 'BT6', 'PVA', 'S12', 'STT', 'BTN', 'KLB', 'DGT', 'V15', 'XMC', 'CBI', 'VC5', 'VES', 'PWA', 'AGF', 'NHH', 'VPG', 'VSC', 'HHP', 'HTN', 'KOS', 'ADG', 'VNG', 'VSC', 'VNG', 'THG', 'BIC', 'PGI', 'CLL', 'VNS', 'LM8', 'SPP', 'FDG', 'NTB', 'NHP', 'CLG', 'LMH', 'ATA', 'VAB', 'D82', 'NDF', 'MSR', 'VLF', 'HDO', 'AMS', 'SSH', 'PVE', 'VNH', 'L44', 'BT6', 'PVA', 'S12', 'STT', 'BTN', 'KLB', 'DGT', 'V15', 'VC5', 'VES', 'PWA', 'AGF', 'VSC']
